In [2]:
# FastText 적용을 위한 corpus 생성 및 크롤링 데이터 학습

from gensim.models import FastText
from tqdm import tqdm


corpus_fname = 'C:/Users/Administrator/Desktop/python/natepann.txt'
model_fname = 'C:/Users/Administrator/Desktop/python/fasttext'


print('corpus 생성')
corpus = [sent.strip().split(" ")for sent in tqdm(open(corpus_fname, 'r', encoding='utf-8').readlines())]

print("학습 중")
model = FastText(corpus, vector_size=100, window=5, min_count=5, workers=4, sg=1)
model.save(model_fname)

print('완료')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
100%|████████████████████████████████████████████████████████████████████████████| 801/801 [00:00<00:00, 135545.77it/s]


corpus 생성
학습 중
완료


In [45]:
# 새로운 용어에 FastText를 적용하여 주변 단어 100개 추출

import pandas as pd

loaded_model = FastText.load("C:/Users/Administrator/Desktop/python/fasttext")
print(loaded_model.wv.vectors.shape)

Similar = []
similar_word_list = list(loaded_model.wv.most_similar("스우파", topn=100)) 

for i in range(len(similar_word_list)):
    temp_str = similar_word_list[i][0].__str__()
    Similar.append(temp_str)
    
print(Similar)


df = pd.DataFrame(Similar) 
df.to_csv('natepann_surrounding_words.csv',header=None,index=False, encoding='utf-8-sig') 

(1671, 100)
['라치카', '보니까', '홀리뱅', '리정', '하니까', '육성재', '모니카', '강다니엘', '이걸', '정리', '시미즈', '어린이집', '남돌', '허니제이', '서울대', '버블', '내년에', '톡선', '걸서위', '이유', '방금', '이달소', '돈이', '이채연', '민아', '둘다', '보아', '아이도', '여자친구', '해주고', '많다', '봐서', '레벨', '프라우드먼', '팬들', '올케언니는', '주연', '안유진', '영훈', '아이는', '노제', '쇼핑몰', '트레저', '이유가', '아들', '나은', '만원을', '블로그', '여돌', '이민혁', '광고', '행복하게', '하였고', '조용히', '로잘린', '안됨', '멜론', '차라리', '분은', '스토리', '추억이', '몇개', '가비', '한국으로', '오늘', '마스카라', '리아', '있던', '올해', '연예인', '위로가', '공주이다', '한국에', '우린', '보낸', '마름', '갈수록', '아이들이', '다이어트', '첫번째', '화장', '저게', '얼굴이', '우주호', '립제이', '마크', '디즈니', '시동생', '서울', '잊을', '있도록', '유튜브', '브랜드', '엠마', '피부', '실력', '미친', '사건', '노래를', '다르게']


In [3]:
# 주변 단어 100개 중 knu 사전에 존재하는 주변 용어만 찾기

import pandas as pd

def fileopen(data):
    
    #파일 불러오기 
    with open(data, 'r', encoding='UTF8') as file:
        
        text = file.read()
        
        splitdata = text.split()
        
        #리스트의 중첩된 부분 삭제하기 
        splitdata = list(dict.fromkeys(splitdata))
 
    return splitdata

if __name__ == '__main__':
 
    #파일1번 불러오기
    NewList = fileopen('natepann_surrounding_words.txt')
    
    #파일2번 불러오기
    NewList1 = fileopen('SentiWord_Dict.txt')
 
    dif1 = list(set(NewList1) - set(NewList))
    dif2 = list(set(NewList) - set(NewList1))
    result = list((set(NewList)-set(dif1))-set(dif2))
    
    
    # 두 파일중 공통된 요소 출력 
    print(result)
    
df = pd.DataFrame(result) 
df.to_csv('natepann_inKNU.txt',header=None,index=False) 

['보기', '많다', '않는', '여자', '보며', '얼굴을', '전부', '좋아하는', '내용을', '쓰고', '커서', '살이', '얼굴이', '중요한', '되어', '좋아하고', '사이가']


In [70]:
# Txt 파일 Knu 사전 Csv 파일로 변환

import pandas as pd
file = pd.read_csv('SentiWord_Dict.txt', delimiter = '\t')

file.to_csv('SentiWord_Dict.csv',index=False, encoding='utf-8-sig') 

In [97]:
# Knu 사전에서 주변 단어가 포함된 키워드들을 찾고, 이들의 극성값을 통계 내서 제공

from IPython.core.display import display, HTML
from konlpy.tag import Okt
import pandas as pd

#키워드에 해당하는 행 출력, 추후 iteration 돌며 키워드 모두 test 할 것
key = '많다|않는|여자|보며|얼굴을|좋아하는|커서|얼굴이|중요한|좋아하고|사이가'

final = pd.DataFrame(columns = ['contents'])

f = pd.read_csv('SentiWord_Dict.csv')

is_key = f['키워드'].str.contains(key, na=False)
f_key = f[is_key]
#f_key = f_key.reset_index()
#f_key

file2 = pd.DataFrame(f_key) 
file2.to_csv('SentiWord_Dict.csv',index=False, encoding='utf-8-sig')

print(f_key)
print("\n새로운 용어의 극성 추정값은 다음과 같습니다:") 
print(file2['극성값'].mean())

          키워드  극성값
0      가장 중요한  1.0
1       겁이 많다 -2.0
2   경망스러운 여자를 -1.0
3   관심이나 좋아하는  2.0
4      굽히지 않는  0.0
..        ...  ...
60    창백한 얼굴을 -1.0
61     충격이 커서 -1.0
62      키가 커서  1.0
63      하지 않는 -1.0
64     현숙한 여자  1.0

[65 rows x 2 columns]

새로운 용어의 극성 추정값은 다음과 같습니다:
0.046153846153846156
